In [ ]:
# 📦 Setup
import sys
sys.path.append("../src")

import torch
from src.utils import load_trained_model
from src.dataset_loader import get_dataloader
from src.evaluate import evaluate_model

# ⚙️ Configurazione
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_trained_model("results/trained_model.pth", num_classes=3, device=device)

# 📥 Validation set
_, val_loader = get_dataloader(batch_size=32)

# 📊 Evaluation
metrics = evaluate_model(model, val_loader)

# 📈 Mostra metriche nel notebook
for k, v in metrics.items():
    print(f"{k.capitalize()}: {v:.4f}")
